## Getting Started 

[Kloppy](https://kloppy.pysport.org/) by [PySport](https://pysport.org/) is _the_ industry standard open-source fooball data standardization package. Kloppy simplifies football data processing by offering a single place to [**load**](https://kloppy.pysport.org/user-guide/loading-data/skillcorner/), [**filter**](https://kloppy.pysport.org/user-guide/getting-started/#filtering-data), [**transform**](https://kloppy.pysport.org/user-guide/transformations/coordinates/) and [**export**](https://kloppy.pysport.org/user-guide/exporting-data/) your football data in a standardized way. 

We can use Kloppy to directly load data from GitHub (see below).

---

### Before we get started:
- Download Python3.11+ if you don't have it already.
- Make a virtual environment to store and install all the Python packages related to this project.
- Activate the virtual environment (select it as a Kernel for this Jupyter Notebook)
- ⚠️ [Install Kloppy](https://kloppy.pysport.org/user-guide/installation/) (`pip install kloppy>=3.18.0`) into the virtual environment

---

### Kloppy

Using Kloppy we can now easily load the tracking data directly from GitHub using the URL to the raw files.




In [36]:
from kloppy import skillcorner

match_id = 1886347

dataset = skillcorner.load_open_data(
    match_id=match_id,
    # Optional Parameters
    coordinates="skillcorner",  # or specify a different coordinate system
    sample_rate=10,  # or specify a different sampling rate
)

c:\Users\g4a4b\anaconda3\envs\hack_env\Lib\site-packages\kloppy\infra\serializers\tracking\skillcorner.py:580: UserWarning: Could not determine orientation of dataset, defaulting to NOT_SET
  warnings.warn(


In [62]:
from kloppy import skillcorner
import os
import json

matches_json_path = os.path.join(os.getcwd(), "data/matches.json")

with open(matches_json_path, "r") as f:
    matches_json = json.load(f)

match_id = matches_json[0]["id"]

# # Construct the raw GitHub URL
tracking_data_github_url=f'https://media.githubusercontent.com/media/SkillCorner/opendata/741bdb798b0c1835057e3fa77244c1571a00e4aa/data/matches/{match_id}/{match_id}_tracking_extrapolated.jsonl'
url = f"https://raw.githubusercontent.com/SkillCorner/opendata/master/data/matches/{match_id}/{match_id}_dynamic_events.csv"
dataset = skillcorner.load(
    meta_data = f'https://raw.githubusercontent.com/SkillCorner/opendata/741bdb798b0c1835057e3fa77244c1571a00e4aa/data/matches/{match_id}/{match_id}_match.json',
    raw_data=tracking_data_github_url,
    # Optional arguments,
    coordinates="skillcorner",
    include_empty_frames=True
)

In [63]:
print(dataset.to_df(engine="pandas"))
print(dataset.metadata)
print(dataset.metadata.frame_rate)

       period_id              timestamp  frame_id ball_state  \
0              1 0 days 00:00:08.800000      2598      alive   
1              1 0 days 00:00:08.900000      2599      alive   
2              1        0 days 00:00:09      2600      alive   
3              1 0 days 00:00:09.100000      2601      alive   
4              1 0 days 00:00:09.200000      2602      alive   
...          ...                    ...       ...        ...   
28883          2 0 days 00:51:11.700000     70387      alive   
28884          2 0 days 00:51:11.800000     70388      alive   
28885          2 0 days 00:51:11.900000     70389      alive   
28886          2        0 days 00:51:12     70390      alive   
28887          2 0 days 00:51:12.100000     70391      alive   

       ball_owning_team_id  ball_x  ball_y  ball_z ball_speed  51678_x  ...  \
0                      868   19.88   20.21    0.91       None   -36.20  ...   
1                      868   20.45   20.34    0.58       None   -36.16  .

### Basic Kloppy Operations

Kloppy has many built in fuctionalities, below is a basic example to:
- Transform the coordinate system such that the home team attack from left to right both periods
- Filter out only the first half
- Output to [Polars](https://pola.rs/) dataframe. You can also output to `"pandas"`

In [53]:
df = (
    dataset.transform(
        to_orientation="STATIC_HOME_AWAY"
    )  # Now, all attacks happen from left to right
    .to_df(
        engine="pandas"
    )  # Convert to a Pandas DataFrame
)

In [54]:
print(df.shape)
print(df.columns)

(28888, 137)
Index(['period_id', 'timestamp', 'frame_id', 'ball_state',
       'ball_owning_team_id', 'ball_x', 'ball_y', 'ball_z', 'ball_speed',
       '51678_x',
       ...
       '50951_d', '50951_s', '51714_x', '51714_y', '51714_d', '51714_s',
       '795521_x', '795521_y', '795521_d', '795521_s'],
      dtype='object', length=137)


---

### Metadata

Kloppy also stores all relevant [**Metadata**](https://kloppy.pysport.org/user-guide/concepts/metadata/) which you can use to easily obtain player names, jersey numbers, playing positions etc.

In [69]:
print(dataset.metadata.periods)

[Period(id=1, start_timestamp=datetime.timedelta(seconds=251), end_timestamp=datetime.timedelta(seconds=3021), prev_period=None, next_period=Period(id=2, start_timestamp=datetime.timedelta(seconds=3967), end_timestamp=datetime.timedelta(seconds=7042), prev_period=..., next_period=None)), Period(id=2, start_timestamp=datetime.timedelta(seconds=3967), end_timestamp=datetime.timedelta(seconds=7042), prev_period=Period(id=1, start_timestamp=datetime.timedelta(seconds=251), end_timestamp=datetime.timedelta(seconds=3021), prev_period=None, next_period=...), next_period=None)]


In [8]:
home_team, away_team = dataset.metadata.teams

for player in home_team.players:
    print(f"{player.jersey_no} - {player.name} - {player.starting_position}")

10 - Guillermo Luis May Bartesaghi - Striker
17 - Callan Elliot - Unknown
22 - Jake Brimmer - Striker
15 - Francis De Vries - Unknown
4 - Nando Pijnaker - Left Center Back
23 - Daniel Hall - Right Center Back
6 - Louis Verstraete - Defensive Midfield
14 - Liam  Gillion - Left Wing
25 - Neyder Stiven Moreno Betancur - Right Wing
5 - Tommy Smith - Center Back
9 - Max Mata - Striker
27 - Logan Rogerson - Right Wing
28 - Luis Felipe Gallegos Leiva - Attacking Midfield
3 - Scott Galloway - Unknown
1 - Michael Woud - Unknown
8 - Luis Toomey - Unknown
7 - Cameron Drew Howieson - Unknown
12 - Alex Noah Paulsen - Unknown


---

### Basic Kloppy Functionalities

Below is a non-exhaustive list of other Kloppy functionalities.

- [TrackingDataset](https://kloppy.pysport.org/user-guide/concepts/tracking-data/)
- [Metadata (players, team names etc.)](https://kloppy.pysport.org/user-guide/concepts/metadata/)
- [Coordinate Systems](https://kloppy.pysport.org/user-guide/concepts/coordinates/#built-in-coordinate-systems)
- [Transformations](https://kloppy.pysport.org/user-guide/transformations/coordinates/)
- [Filter](https://kloppy.pysport.org/user-guide/getting-started/#filtering-data)
- [Exporting to pandas / polars DataFrames](https://kloppy.pysport.org/user-guide/exporting-data/dataframes/)

## Existing Open Source Projects
Below is a non-exhaustive list of publicly available **open source** football analytics tools. Feel free to use these as inspiration, build on-top of these tools or develop them further. We advise Analytics Cup participants to not replicate existing work, but rather use it to build new open source tools instead.

**Plotting**

- [`mplsoccer`](https://mplsoccer.readthedocs.io/en/latest/) and `matplotlib` to plot some different configurations of tracking data.
- [`soccer-d3`](https://github.com/probberechts/d3-soccer)
- And check out [Kloppy's Plotting Examples](https://kloppy.pysport.org/user-guide/getting-started/#exec-51--__tabbed_1_2)

**Resources for Positional Tracking Data**
- [floodlight](https://floodlight.readthedocs.io/en/latest/)
- [databallpy](https://databallpy.readthedocs.io/en/latest/?badge=latest)
- [Hyunsung Kim @ KAIST](https://github.com/hyunsungkim-ds)
- [Friends of Tracking](https://github.com/Friends-of-Tracking-Data-FoTD)
- [unravelsports](https://github.com/UnravelSports/UnravelSports)

**Other Football Analytics Resources**
- [socceraction](https://socceraction.readthedocs.io/en/latest/)
- [Footballdata](https://Footballdata.readthedocs.io/en/latest/)
- [ETSY](https://github.com/ML-KULeuven/ETSY) 
- [soccer-xg](https://github.com/ML-KULeuven/soccer_xg)
- [OpenSTARLab](https://openstarlab.readthedocs.io/en/latest/)
- [Soccer Analytics Handbook](https://github.com/devinpleuler/analytics-handbook)
- [penaltyblog](https://pypi.org/project/penaltyblog/)
